In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import xgboost

In [2]:
train_users = pd.read_csv('../cache/train_users.csv')

In [3]:
train_users.fillna(-1, inplace=True)

y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id'], axis=1, inplace=True)
x_train = train_users.values

label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [4]:
train_data = xgboost.DMatrix(x_train, encoded_y_train)

In [5]:
def ndcg5(preds, dtrain, k=5):
    y_true = dtrain.get_label()
    n = len(y_true)
    num_class = preds.shape[1]
    index = np.argsort(preds, axis=1)
    top = index[:, -k:][:,::-1]
    rel = (np.reshape(y_true, (n, 1))==top).astype(int)
    cal_dcg = lambda y: sum((2**y - 1)/np.log2(range(2, k+2)))
    ndcg = np.mean((np.apply_along_axis(cal_dcg, 1, rel)))
    return 'ndcg5', -ndcg

In [6]:
def customized_eval(preds, dtrain):
    labels = dtrain.get_label()
    top = []
    for i in range(preds.shape[0]):
        top.append(np.argsort(preds[i])[::-1][:5])
    mat = np.reshape(np.repeat(labels,np.shape(top)[1]) == np.array(top).ravel(),np.array(top).shape).astype(int)
    score = np.mean(np.sum(mat/np.log2(np.arange(2, mat.shape[1] + 2)),axis = 1))
    return 'ndcg5', score

In [7]:
param = {
    'max_depth': 10,
    'learning_rate': 10,
    'n_estimators': 10,
    'objective': 'multi:softprob',
    'num_class': 3,
    'gamma': 0,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'missing': None,
    'silent': True,
    'nthread': -1,
    'seed': 42
}

num_round = 10
xgboost.cv(param, train_data, num_boost_round=num_round, metrics=['mlogloss'], feval=customized_eval)

,test-ndcg5-mean,test-ndcg5-std,train-ndcg5-mean,train-ndcg5-std
0,0.521822,0.03086,0.845562,0.044428
1,0.521822,0.03086,0.845562,0.044428
2,0.521822,0.03086,0.845562,0.044428
3,0.521822,0.03086,0.845562,0.044428
4,0.521822,0.03086,0.845562,0.044428
5,0.521822,0.03086,0.845562,0.044428
6,0.521822,0.03086,0.845562,0.044428
7,0.521822,0.03086,0.845562,0.044428
8,0.521822,0.03086,0.845562,0.044428
9,0.521822,0.03086,0.845562,0.044428
